# LangChain Expression Language (LCEL)

LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production). To highlight a few of the reasons you might want to use LCEL:

_**First-class streaming support**_ 

When you build your chains with LCEL you get the best possible time-to-first-token (time elapsed until the first chunk of output comes out). For some chains this means eg. we stream tokens straight from an LLM to a streaming output parser, and you get back parsed, incremental chunks of output at the same rate as the LLM provider outputs the raw tokens.

This interface provides two general approaches to stream content:

1. sync stream and async astream: a default implementation of streaming that streams the final output from the chain.
2. async astream_events and async astream_log: these provide a way to stream both intermediate steps and final output from the chain.

In [11]:
import os

#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

In [12]:
from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a brief explaination about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "blackhole"}):
    print(chunk, end="", flush=True)

A black hole is a region in space where gravity is extremely strong, and nothing, not even light, can escape its pull. It is formed when a massive star collapses under its own gravitational force after burning out its nuclear fuel. As the star collapses, its size decreases while its mass remains the same, causing a tremendous increase in density. This immense concentration of mass creates a gravitational field so intense that it traps everything within its event horizon, the boundary beyond which nothing can escape. Black holes are invisible, but their presence can be inferred by the effects they have on surrounding matter and light. They continue to be a subject of intense scientific research and fascination due to their mysterious nature.

_**Async support**_ 

Any chain built with LCEL can be called both with the synchronous API (eg. in your Jupyter notebook while prototyping) as well as with the asynchronous API (eg. in a LangServe server). This enables using the same code for prototypes and in production, with great performance, and the ability to handle many concurrent requests in the same server.


In [14]:
for s in chain.stream({"topic": "northern lights"}):
    print(s, end="", flush=True)

The Northern Lights, also known as Aurora Borealis, are natural light displays that occur in the sky, primarily in the polar regions. They are caused by the interaction between the Earth's magnetic field and charged particles from the Sun. 

When the Sun emits a large amount of charged particles called solar wind, they travel towards the Earth. As they approach our planet, they interact with the magnetosphere, which is the region surrounding the Earth controlled by its magnetic field. 

The charged particles from the Sun are trapped and directed towards the poles by the Earth's magnetic field. As they collide with molecules and atoms in the Earth's atmosphere, they release energy in the form of colorful lights. These lights can appear in various colors, including green, pink, red, purple, and blue.

The Northern Lights are typically observed in high-latitude regions such as Alaska, Canada, Scandinavia, and Iceland. They often appear as shimmering curtains or waves of lights dancing acr

In [15]:
async for s in chain.astream({"topic": "String theory"}):
    print(s, end="", flush=True)

String theory is a theoretical framework in physics that attempts to explain the fundamental nature of particles and the forces that govern them. It proposes that all particles in the universe are not point-like particles but instead tiny, vibrating strings. These strings can vibrate in different ways, giving rise to different particle properties such as mass and charge.

String theory also suggests that there are additional dimensions beyond the three spatial dimensions we are familiar with. These extra dimensions are compactified and curled up at microscopic scales, making them undetectable in our everyday experience. The theory proposes that the universe we observe is just one of many possible configurations of these extra dimensions.

One of the most intriguing aspects of string theory is its potential to unify all the fundamental forces of nature, including gravity. By incorporating gravity into the framework of quantum mechanics, string theory offers a promising approach to resol

In [16]:
chain.invoke({"topic": "Global warming and its effects"})

"Global warming refers to the long-term increase in Earth's average surface temperature due to human activities, primarily the emission of greenhouse gases into the atmosphere. These gases, such as carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O), trap heat from the sun and cause the planet to warm up. The main contributors to global warming are the burning of fossil fuels (coal, oil, and natural gas), deforestation, and industrial processes.\n\nThe effects of global warming are far-reaching and pose significant risks to both the environment and human societies. Some of the key impacts include:\n\n1. Rising temperatures: Global warming leads to higher average temperatures worldwide, causing heatwaves and extreme weather events to become more frequent and intense.\n\n2. Melting ice: As temperatures increase, glaciers and polar ice caps melt, leading to rising sea levels. This poses a threat to coastal areas, increases the risk of flooding, and endangers low-lying islands.\n\

In [17]:
chain.batch([{"topic": "big bang"}, {"topic": "how the universe created"}])

['The Big Bang theory is the prevailing scientific explanation for the origin of the universe. It suggests that around 13.8 billion years ago, the entire universe was condensed into an incredibly hot and dense point called a singularity. This singularity suddenly expanded, resulting in an explosion of energy and the formation of all matter and energy that exist today.\n\nAs the universe rapidly expanded, it cooled down, allowing subatomic particles like protons and neutrons to form. Eventually, these particles combined to create atoms, which further coalesced into stars, galaxies, and other structures we observe in the universe.\n\nThe Big Bang theory is supported by various lines of evidence, such as the observed expansion of the universe, the detection of cosmic microwave background radiation, and the abundance of light elements like hydrogen and helium. However, there are still many unanswered questions about the precise mechanisms and events that occurred during the early moments o

In [18]:
await chain.ainvoke({"topic": "E=mc2"})

'E=mc2 is a famous equation derived by Albert Einstein in his theory of relativity. It states that energy (E) is equal to the mass (m) of an object multiplied by the speed of light (c) squared. This equation shows that there is a direct relationship between mass and energy, suggesting that mass can be converted into energy and vice versa. It also implies that a small amount of mass can produce a large amount of energy, as the speed of light is a very large number. The equation has profound implications for understanding the behavior of matter and energy in the universe.'

_**Optimized parallel execution**_ 

Whenever your LCEL chains have steps that can be executed in parallel (eg if you fetch documents from multiple retrievers) we automatically do it, both in the sync and the async interfaces, for the smallest possible latency.

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(model = os.environ["LLM_EMBEDDING_MODEL"],)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embeddings
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
response = retrieval_chain.invoke("where did harrison work?")

In [21]:
response

'Harrison worked at Kensho.'

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

In [24]:
response = map_chain.invoke({"topic": "Steve Jobs"})

In [34]:
response

{'joke': AIMessage(content='Why did Steve Jobs become a chef?\n\nBecause he knew how to cook up some amazing Apple pies!', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 14, 'total_tokens': 34}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-8523f7bf-4333-4ebf-a15d-f1917f062c4f-0'),
 'poem': AIMessage(content="Innovation's spark, a visionary's quest,\nSteve Jobs, forever changing th

In [33]:
print(response['joke'].content)

Why did Steve Jobs become a chef?

Because he knew how to cook up some amazing Apple pies!


In [35]:
print(response['poem'].content)

Innovation's spark, a visionary's quest,
Steve Jobs, forever changing the world's behest.


In [36]:
%%timeit

joke_chain.invoke({"topic": "bear"})

542 ms ± 48.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit

poem_chain.invoke({"topic": "bear"})

574 ms ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit

map_chain.invoke({"topic": "bear"})

646 ms ± 25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


_**Retries and fallbacks**_

Configure retries and fallbacks for any part of your LCEL chain. This is a great way to make your chains more reliable at scale. We’re currently working on adding streaming support for retries/fallbacks, so you can get the added reliability without any latency cost.

1. Fallback for LLM API Errors - A request to an LLM API can fail for a variety of reasons - the API could be down, you could have hit rate limits, any number of things. Therefore, using fallbacks can help protect against these types of things.
2. Fallback for Sequences - ChatOpenAI and then normal OpenAI (which does not use a chat model). Because OpenAI is NOT a chat model, you likely want a different prompt.
3. Fallback for Long Inputs - This model's maximum context length is 4097 tokens. However, your messages resulted in 12012 tokens. Please reduce the length of the messages.


4. Fallback to Better Mo - Models like GPT-3.5 can do this okay, but sometimes struggle. This naturally points to fallbacks - we can try with GPT-3.5 (faster, cheaper), but then if parsing fails we can use GPT-4.del

In [45]:
#Lets see an example for Fallback for Sequences

# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = AzureChatOpenAI(
    model_name="gpt-free",
    api_version="2023-07-01-preview"
)
bad_chain = chat_prompt | chat_model | StrOutputParser()

In [41]:
# Now lets create a chain with the normal OpenAI model
from langchain_core.prompts import PromptTemplate
#from langchain_openai import OpenAI

# Import Azure OpenAI
from langchain_openai import AzureOpenAI



In [42]:
# Create an instance of Azure OpenAI
# Replace the deployment name with your own
llm = AzureOpenAI(
    model_name="gpt-35-turbo-16k",
    api_version="2023-07-01-preview")

In [43]:
prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)
#llm = AzureOpenAI()
good_chain = prompt | llm

In [46]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

_**Access intermediate results**_ 

For more complex chains it’s often very useful to access the results of intermediate steps even before the final output is produced. This can be used to let end-users know something is happening, or even just to debug your chain. You can stream intermediate results, and it’s available on every LangServe server.



_**Input and output schemas**_ 

Input and output schemas give every LCEL chain Pydantic and JSONSchema schemas inferred from the structure of your chain. This can be used for validation of inputs and outputs, and is an integral part of LangServe.

In [52]:
# The input schema of the chain is the input schema of its first part, the prompt.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'animal': {'title': 'Animal', 'type': 'string'}}}

In [53]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'animal': {'title': 'Animal', 'type': 'string'}}}

In [54]:
model.input_schema.schema()

{'title': 'AzureChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'i

In [55]:
# The output schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

_**Seamless LangSmith tracing**_

As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximum observability and debuggability.

_**Seamless LangServe deployment**_

Any chain created with LCEL can be easily deployed using LangServe.